In [1]:
import os
import sys
sys.modules.pop('src.sentinel_preprocessing', None)
sys.modules.pop('src.sam_satellite_processor', None)

sys.path.append("..")
from src.sentinel_preprocessing import preprocess_imagery
from src.sentinel_preprocessing import build_color_from_JP2
from src.sentinel_preprocessing import split_image_in_tiles
from src.sam_satellite_processor import segment_satellite_imagery
from src.sam_satellite_processor import segment_single_image

In [ ]:
pleiade_folder = "/home/teulade/images/images_pleiades/20160609T104841_PLD_SEN_PMS_1816164101-001"
pleiade_jp2 = os.path.join(pleiade_folder,"IMG_PHR1A_PMS_201606091048415_SEN_1816164101-001_R1C1.JP2")
# Cell with preprocessing
color_type = 'rgb'  # or 'rgb'
#grid_size = 33  # or your preferred size

# Choose one of these approaches:
# preprocess_imagery(pleiade_jp2, color_type=color_type)
# or
# composite_image = build_color_from_JP2(pleiade_jp2, color_type=color_type)
composite_image = f"{pleiade_folder}/rgb/pleiades_composite.tif"
split_image_in_tiles(input_file=composite_image, grid_size=33)

In [2]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

sam_checkpoint = "/home/teulade/satellite-sam-segmentation/models/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.6,
    stability_score_thresh=0.6,
    crop_nms_thresh=0,
    crop_overlap_ratio=1,
    crop_n_layers=1,
    min_mask_region_area=50,
)

/.conda/envs/teulade-sat_proj_env/lib/python3.11/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [ ]:
# segment_satellite_imagery(pleiade_folder, mask_generator, color_type="rgb",n_samples=10)

segment_single_image("/home/teulade/images/images_pleiades/20160609T104841_PLD_SEN_PMS_1816164101-001/rgb/tiles_33x33/tile_9_9.tif", "/home/teulade/images/tmp_output", mask_generator)

In [ ]:
import geopandas as gpd
import rasterio
from shapely.geometry import Polygon

# Read the saved parquet file with the segments
segments_path = f"{pleiade_folder}/shapefiles_10000tiles_32points/polygons.parquet"
gdf = gpd.read_parquet(segments_path)

# Get the centroid of our data to determine the UTM zone
center_lon = gdf.geometry.centroid.x.mean()
center_lat = gdf.geometry.centroid.y.mean()

# Calculate UTM zone number
utm_zone = int((center_lon + 180) / 6) + 1
epsg_code = 32600 + utm_zone  # Northern hemisphere
if center_lat < 0:
    epsg_code = 32700 + utm_zone  # Southern hemisphere

print(f"Using UTM Zone {utm_zone} (EPSG:{epsg_code}) for area calculations")


# Project to appropriate UTM zone for accurate area calculation
gdf_projected = gdf.to_crs(epsg=epsg_code)
total_segments_area = gdf_projected.geometry.area.sum()


# Get the tile bounds from the original image
tile_path = f"{pleiade_folder}/output_nrg.tif"
with rasterio.open(tile_path) as src:
    bounds = src.bounds
    tile_geom = Polygon([
        (bounds.left, bounds.bottom),
        (bounds.left, bounds.top),
        (bounds.right, bounds.top),
        (bounds.right, bounds.bottom)
    ])
    tile_crs = src.crs

# Calculate total tile area
tile_gdf = gpd.GeoDataFrame({'geometry': [tile_geom]}, crs=tile_crs)
tile_gdf_projected = tile_gdf.to_crs(epsg=epsg_code)
total_tile_area = tile_gdf_projected.geometry.area.iloc[0]

# Calculate and print statistics
coverage_percentage = (total_segments_area / total_tile_area) * 100

print(f"Number of segments: {len(gdf)}")
print(f"Total segment area: {total_segments_area:.2f} m²")
print(f"Total tile area: {total_tile_area:.2f} m²")
print(f"Coverage percentage: {coverage_percentage:.2f}%")